# Transition-based arc-eager unlabeled dependency parser for Ukrainian

## Read the data

Useful links:
* [UD corpus for Ukrainian](https://github.com/UniversalDependencies/UD_Ukrainian-IU/)
* [Easy-to-use library for parsing UD](https://github.com/EmilStenstrom/conllu)

In [25]:
from collections import OrderedDict
from conllu import parse
from enum import Enum

In [26]:
PATH = "UD_Ukrainian-IU"

with open(PATH + "/uk_iu-ud-train.conllu", "r", encoding='utf-8') as f:
    dev_trees = parse(f.read())

with open(PATH + "/uk_iu-ud-train.conllu", "r", encoding='utf-8') as f:
    train_trees = parse(f.read())

with open(PATH + "/uk_iu-ud-dev.conllu", "r", encoding='utf-8') as f:
    test_trees = parse(f.read())

In [27]:
print(len(dev_trees))
print(len(train_trees))
print(len(test_trees))

5496
5496
672


In [28]:
tree=dev_trees[0]
print(tree)
for node in tree:
    print(node)

TokenList<У, домі, римського, патриція, Руфіна, була, прегарна, фреска, ,, зображення, Венери, та, Адоніса, .>
{'id': 1, 'form': 'У', 'lemma': 'у', 'upos': 'ADP', 'xpos': 'Spsl', 'feats': {'Case': 'Loc'}, 'head': 2, 'deprel': 'case', 'deps': [('case', 2)], 'misc': {'Id': '0003', 'LTranslit': 'u', 'Translit': 'U'}}
{'id': 2, 'form': 'домі', 'lemma': 'дім', 'upos': 'NOUN', 'xpos': 'Ncmsln', 'feats': {'Animacy': 'Inan', 'Case': 'Loc', 'Gender': 'Masc', 'Number': 'Sing'}, 'head': 6, 'deprel': 'obl', 'deps': [('obl', 6)], 'misc': {'Id': '0004', 'LTranslit': 'dim', 'Translit': 'domi'}}
{'id': 3, 'form': 'римського', 'lemma': 'римський', 'upos': 'ADJ', 'xpos': 'Ao-msgf', 'feats': {'Case': 'Gen', 'Gender': 'Masc', 'Number': 'Sing'}, 'head': 4, 'deprel': 'amod', 'deps': [('amod', 4)], 'misc': {'Id': '0005', 'LTranslit': 'rymśkyj', 'Translit': 'rymśkoho'}}
{'id': 4, 'form': 'патриція', 'lemma': 'патрицій', 'upos': 'NOUN', 'xpos': 'Ncmsgy', 'feats': {'Animacy': 'Anim', 'Case': 'Gen', 'Gender': 'M

## Design actions and the oracle

We will be using a static oracle that reproduces a single valid order of actions.

In [29]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"

In [30]:
def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [31]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

def trace_actions(tree, log=True):
    """
    Try out the oracle to verify it's returning the right actions.
    """
    stack, queue, relations = [ROOT], tree[:], []
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        if log:
            print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
            print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
            print("Relations:", relations)
            print(action)
            print("========================")
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    if log:
        print("Gold relations:")
        print([(node["id"], node["head"]) for node in tree])
        print("Retrieved relations:")
        print(sorted(relations))

trace_actions(tree)

Stack: ['ROOT_0']
Queue: ['У_1', 'домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'У_1']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.LEFT
Stack: ['ROOT_0']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2']
Queue: ['римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2', 'римського_3']
Queue: ['патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'В

## Feature extraction

Reference: [Dependency Parsing by Kübler, McDonald, and Nivre](https://books.google.com.ua/books?id=k3iiup7HB9UC&pg=PA21&hl=uk&source=gbs_toc_r&cad=4#v=onepage&q&f=false)

In [32]:
# 111126
def extract_features(stack, queue):
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
    if len(stack) > 1:
        features["s1-tag"] = stack[-2]["upostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    return features

### The first iteration is increasing the number of  additional features from 111126 to 617023.

In [44]:
# 617023
def extract_features_extended(stack, queue):
    features = dict()
    if len(stack) > 0:
        for i in range(len(stack)):
            stack_top = stack[-i-1]
            features["s0{}-word".format(i)] = stack_top["form"]
            features["s0{}-lemma".format(i)] = stack_top["lemma"]
            features["s0{}-tag".format(i)] = stack_top["upostag"]
            if stack_top["feats"]:
                for key, val in stack_top["feats"].items():
                    features["s0{}-".format(i) + key] = val
            if stack_top["misc"]:
                for key, val in stack_top["misc"].items():
                    features["s0{}-".format(i) + key] = val
    if len(stack) > 1:
        for i in range(1, len(stack)):
            features["s0{}-tag".format(i)] = stack[-2]["upostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        if queue[0]["feats"]:
            for key, val in queue[0]["feats"].items():
                features["q0-" + key] = val
        if stack_top["misc"]:
            for key, val in stack_top["misc"].items():
                features["s0{}-".format(i) + key] = val
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-lemma"] = queue_next["lemma"]
        features["q1-tag"] = queue_next["upostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    return features

## Prepare train and test data

In [34]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

In [45]:
def get_data(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []

    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        features.append(extract_features_extended(stack, queue))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    return features, labels

In [46]:
# A simple hack would be to check the type of the node id

train_features, train_labels = [], []
for tree in train_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

190298 190298


In [47]:
# Test data

test_features, test_labels = [], []
for tree in test_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    test_features += tree_features
    test_labels += tree_labels

print(len(test_features), len(test_labels))

25820 25820


## Train a classifier

In [48]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
# https://analyticsindiamag.com/a-beginners-guide-to-scikit-learns-mlpclassifier/

In [ ]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)

print("\nTotal number of features: ", len(vec.get_feature_names()))

In [ ]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

# print(len(train_features_vectorized.toarray()), len(test_features_vectorized.toarray()))

In [ ]:
lrc = LogisticRegression(random_state=42, solver="saga", multi_class="multinomial", max_iter=600, verbose=1)
lrc.fit(train_features_vectorized, train_labels)

In [ ]:
# Basic reults:
#               precision    recall  f1-score   support

#         left       0.86      0.87      0.86      6371
#       reduce       0.85      0.78      0.81      6875
#        right       0.75      0.79      0.77      5996
#        shift       0.85      0.87      0.86      6578

#     accuracy                           0.83     25820
#    macro avg       0.83      0.83      0.83     25820
# weighted avg       0.83      0.83      0.83     25820

#### The best model accuracy

In [ ]:
# Result with extended list of features:
predicted = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

In [ ]:
# Iteration 2:
# Using Multi-layer Perceptron classifier

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=12, verbose=True,
                    learning_rate_init=0.1, max_iter=600)

mlp.fit(train_features_vectorized, train_labels)
predicted = mlp.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

In [ ]:
#               precision    recall  f1-score   support

#         left       0.66      0.95      0.78      6371
#       reduce       0.89      0.60      0.71      6875
#        right       0.75      0.76      0.76      5996
#        shift       0.90      0.81      0.85      6578

#     accuracy                           0.78     25820
#    macro avg       0.80      0.78      0.78     25820
# weighted avg       0.80      0.78      0.77     25820

In [ ]:
# Iteration 3:
# Using Naive Bayes Classifier

In [ ]:
mnnb = MultinomialNB()
mnnb.fit(train_features_vectorized, train_labels)
predicted_mnnb = mnnb.predict(test_features_vectorized)
print(classification_report(test_labels, predicted_mnnb))

In [ ]:
#               precision    recall  f1-score   support

#         left       0.70      0.82      0.76      6371
#       reduce       0.72      0.55      0.62      6875
#        right       0.53      0.69      0.60      5996
#        shift       0.83      0.66      0.74      6578

#     accuracy                           0.68     25820
#    macro avg       0.70      0.68      0.68     25820
# weighted avg       0.70      0.68      0.68     25820

## Calculate the unlabeled attachment score
UAS - the percentage of words in an input that are assigned the correct head.

In [ ]:
def dep_parse(sentence, oracle, vectorizer, log=True):
    stack, queue, relations = [ROOT], sentence[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features_extended(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
            # actual parsing
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

In [49]:
total, tp, full_match = 0, 0, 0
for tree in test_trees:
    tree = [t for t in tree if type(t["id"])==int]
    golden = [(node["id"], node["head"]) for node in tree]
    predicted = dep_parse(tree, lrc, vec, log=False)
    total += len(tree)
    tp += len(set(golden).intersection(set(predicted)))
    if set(golden) == set(predicted):
        full_match += 1

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))
print("Full match:", round(full_match/len(test_trees), 2))


Total number of features:  616765


In [50]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

# print(len(train_features_vectorized.toarray()), len(test_features_vectorized.toarray()))

In [51]:
lrc = LogisticRegression(random_state=42, solver="saga", multi_class="multinomial", max_iter=600, verbose=1)
lrc.fit(train_features_vectorized, train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 298 epochs took 191 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.2min finished


LogisticRegression(max_iter=600, multi_class='multinomial', random_state=42,
                   solver='saga', verbose=1)

In [52]:
# Basic reults:
#               precision    recall  f1-score   support

#         left       0.86      0.87      0.86      6371
#       reduce       0.85      0.78      0.81      6875
#        right       0.75      0.79      0.77      5996
#        shift       0.85      0.87      0.86      6578

#     accuracy                           0.83     25820
#    macro avg       0.83      0.83      0.83     25820
# weighted avg       0.83      0.83      0.83     25820

#### The best model accuracy

In [53]:
# Result with extended list of features:
predicted = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

              precision    recall  f1-score   support

        left       0.87      0.89      0.88      6371
      reduce       0.86      0.81      0.83      6875
       right       0.77      0.80      0.79      5996
       shift       0.85      0.86      0.86      6578

    accuracy                           0.84     25820
   macro avg       0.84      0.84      0.84     25820
weighted avg       0.84      0.84      0.84     25820



In [55]:
# Iteration 2:
# Using Multi-layer Perceptron classifier

In [58]:
mlp = MLPClassifier(hidden_layer_sizes=12, verbose=True,
                    learning_rate_init=0.1, max_iter=600)

mlp.fit(train_features_vectorized, train_labels)
predicted = mlp.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

Iteration 1, loss = 0.85093716
Iteration 2, loss = 0.77250871
Iteration 3, loss = 0.77026892
Iteration 4, loss = 0.77419655
Iteration 5, loss = 0.77052740
Iteration 6, loss = 0.77731507
Iteration 7, loss = 0.78225686
Iteration 8, loss = 0.77162010
Iteration 9, loss = 0.78228451
Iteration 10, loss = 0.78471614


d:\ucuniversity\nlp\module2\rulebased\.env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:587: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


              precision    recall  f1-score   support

        left       0.66      0.95      0.78      6371
      reduce       0.89      0.60      0.71      6875
       right       0.75      0.76      0.76      5996
       shift       0.90      0.81      0.85      6578

    accuracy                           0.78     25820
   macro avg       0.80      0.78      0.78     25820
weighted avg       0.80      0.78      0.77     25820



In [ ]:
#               precision    recall  f1-score   support

#         left       0.66      0.95      0.78      6371
#       reduce       0.89      0.60      0.71      6875
#        right       0.75      0.76      0.76      5996
#        shift       0.90      0.81      0.85      6578

#     accuracy                           0.78     25820
#    macro avg       0.80      0.78      0.78     25820
# weighted avg       0.80      0.78      0.77     25820

In [54]:
# Iteration 3:
# Using Naive Bayes Classifier

In [60]:
mnnb = MultinomialNB()
mnnb.fit(train_features_vectorized, train_labels)
predicted_mnnb = mnnb.predict(test_features_vectorized)
print(classification_report(test_labels, predicted_mnnb))

              precision    recall  f1-score   support

        left       0.70      0.82      0.76      6371
      reduce       0.72      0.55      0.62      6875
       right       0.53      0.69      0.60      5996
       shift       0.83      0.66      0.74      6578

    accuracy                           0.68     25820
   macro avg       0.70      0.68      0.68     25820
weighted avg       0.70      0.68      0.68     25820



In [ ]:
#               precision    recall  f1-score   support

#         left       0.70      0.82      0.76      6371
#       reduce       0.72      0.55      0.62      6875
#        right       0.53      0.69      0.60      5996
#        shift       0.83      0.66      0.74      6578

#     accuracy                           0.68     25820
#    macro avg       0.70      0.68      0.68     25820
# weighted avg       0.70      0.68      0.68     25820

## Calculate the unlabeled attachment score
UAS - the percentage of words in an input that are assigned the correct head.

In [19]:
def dep_parse(sentence, oracle, vectorizer, log=True):
    stack, queue, relations = [ROOT], sentence[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features_extended(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
            # actual parsing
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

In [20]:
total, tp, full_match = 0, 0, 0
for tree in test_trees:
    tree = [t for t in tree if type(t["id"])==int]
    golden = [(node["id"], node["head"]) for node in tree]
    predicted = dep_parse(tree, lrc, vec, log=False)
    total += len(tree)
    tp += len(set(golden).intersection(set(predicted)))
    if set(golden) == set(predicted):
        full_match += 1

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))
print("Full match:", round(full_match/len(test_trees), 2))

Total: 12574
Correctly defined: 8857
UAS: 0.7
Full match: 0.11


## Find non-projective trees

In [21]:
def is_non_projective(tree):
    relations = [[i['id'], i['head']] for i in tree if type(i["id"])==int]
    for rel in relations:
        for ref_rel in relations:
            a, c = sorted(rel)
            b, d = sorted(ref_rel)
            if a < b and b < c and c < d:
                return True
    return False

total_non_pr = 0
tree_ids = []
for i in range(len(train_trees)):
    if is_non_projective(train_trees[i]):
        total_non_pr += 1
        tree_ids.append(i)

print("The percentage of non-projective trees is {} ({} out of {}).".
      format(round(total_non_pr * 100 / len(train_trees), 2), total_non_pr, len(train_trees)))

print("IDs:", tree_ids[:10])

The percentage of non-projective trees is 7.99 (439 out of 5496).
IDs: [4, 9, 13, 19, 22, 28, 29, 33, 34, 43]
